### Quantities & Prices



Suppose for some good we observe data on total quantity $q$ exchanged
between buyers & sellers, and the prices $p$ at which these exchanges
took place.  

The following code creates a data generating process for $(q,p)$ based
on Goldberger (1972), who in turn is describing the work of Sewall
Wright (1934).  The demand-supply system is
$$
   q_D = \alpha p + u\qquad q_S = \beta p + v\qquad q_D = q_S,
$$
where $(u,v)$ are unobserved shocks to demand and supply,
respectively. 



In [1]:
import statistics as stats
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid

# Structural parameters;
α = -1
β = 2
mu_D = 2
mu_S = -1
sigma_D = 1/2
sigma_S = 1/3

# u,v assumed independent
u = iid.norm(loc=mu_D, scale=sigma_D)  # Demand shocks
v = iid.norm(loc=mu_S, scale=sigma_S)  # Supply shocks

# Reduced form coefficients
π = [[-β/(α - β), -1/(α - β)],
     [ α/(α - β), 1/(α - β)]]

# Generate N realizations of system
# Outcomes Y have columns (q,p)
N = 10000

# Arrange shocks into an Nx2 matrix
U = np.c_[u.rvs(N), v.rvs(N)]

# Matrix product gives [q,p]; label by putting into df
df = pd.DataFrame(U@π,columns=['q','p'])
Udf = pd.DataFrame(U,columns=['u','v']) # For future reference

#### Controlling Price



Consider the question: what would expected demand be if we *fixed*
    the price at $p_0$?  Expected supply?
    
If I were feeling energized and had my math brain on, I would take the expectation of our supply and demand functions to yield functions of p, and then solve for that value. However, it's the afternoon, so instead I will generate a large number of sample estimates and take the sample average.

In [2]:
p_0 = 1.5
q_D = α * p_0 + Udf['u']
q_S = β * p_0 + Udf['v']
sample_quantities = np.column_stack((q_D, q_S))
np.apply_along_axis(stats.mean, 0, sample_quantities)

array([0.50217967, 2.00224257])

The result above gives the expected value of $q_D$ and $q_S$ if we set $p = p_0 = 1.5$. By choosing a different value of $p_0$, we can approximate supply and demand in a similar manner. 

Okay, on second thought, here's a crack at the expectation. 

$$
\begin{aligned}
E[q_D] 
&= \int q_D(p, u) dF_u \\
&= \int (\alpha p + u) dF_u \\
&= \alpha p \int dF_u + \int u dF_u \\
&= \alpha p + E[u]
\end{aligned}
$$

And a similar process derives $E[q_S] = \beta p + E[v]$. So we can now find the expected values given our $p = p_0 = 1.5$. 

In [6]:
e_q_D = α * p_0 + mu_D
e_q_S = β * p_0 + mu_S
(e_q_D, e_q_S)

(0.5, 2.0)

#### Average Causal Effect of a Change in Price



This approach I'm not actually super sure how to proceed with; we mostly just handwaved at it in class. Maybe we can talk thru it in a bit more detail? 

#### Price Change /Ceteris Paribus/



Suppose we *observe* prices and quantities $(p_0,q_0)$.  How *would*
    we expect the quantity demanded to change if prices were instead
    fixed at $p_1$, *ceteris paribus*?

If we observe a fixed market equilibrium, given we know the functional form of supply and demand, we can use these values of $p_0$ and $q_0$ to determine $u_0$ and $v_0$. If we then fix a new price $p_1$, we can then solve supply and demand with our fixed price, $u_0$, and $v_0$. Suppose our realization of $p_0$ and $q_0$ yields the following values for $u_0$, and $v_0$:

In [4]:
Udf.loc[1, ]

u    1.709908
v   -0.907506
Name: 1, dtype: float64

If we then let $p_1 = 1.5$, we can solve for our new demand and supply quantities:

In [5]:
p_1 = 1.5
q_D_cp = α * p_1 + Udf.loc[1, 'u']
q_S_cp = β * p_1 - Udf.loc[1, 'v']
(q_D_cp, q_S_cp)

(0.2099081624232264, 3.907506128534863)